In [38]:
import argparse
import random
import torch
from torch import nn
import numpy as np
from tqdm import tqdm
import functools
import os
import cProfile
import copy
import time
import sys
import json
import subprocess
from torch import nn
from sklearn.model_selection import train_test_split
import timm
import pandas
import seaborn
import matplotlib.pyplot
import numpy as np
from tqdm import tqdm
from collections import Counter
import math
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import numpy as np
import csv
import argparse
import sys
import ast
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from typing import Tuple, Dict



import wandb
from sklearn.metrics.pairwise import cosine_similarity

from torch.utils.data import DataLoader

from src.backbone import get_backbone
from src.modules import CosineLinear
from src.moe_seed import MoE_SEED
from src.data import (
    CILDataManager,
    DILDataManager,
    get_dataset,
    DATASET_MAP,
    make_test_transform_from_args,
    make_train_transform_from_args,
    update_transforms,
)
from src.logging import Logger, WandbLogger, ConsoleLogger, TQDMLogger
from torch.utils.data import Subset

from src.support_functions import check_gpu_memory, shrink_dataset, display_profile, log_gpustat, optimize_args



# Important! Only one dataset at a time

In [5]:
DATASET = "vtab"
CUDA_VISIBLE_DEVICES = "4"

## Get dataset or class features

In [39]:

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def setup_logger(args):
    Logger.instance().add_backend(ConsoleLogger())
    if args.wandb_project is not None:
        Logger.instance().add_backend(
            WandbLogger(args.wandb_project, args.wandb_entity, args)
        )



def update_args(args):
    assert args.k >= 1 and args.k <= 12
    args.intralayers = [f"blocks.{11 - i}" for i in range(args.k)]

    args.aug_normalize = bool(args.aug_normalize)

    args.target_size = 224
    
    dataset_T_map = {
        "dil_imagenetr": {"T": 15, "moe_max_experts": 2},
        "limited_domainnet": {"T": 6, "moe_max_experts": 3},
        "vtab": {"T": 5, "moe_max_experts": 3},
        "cddb": {"T": 5, "moe_max_experts": 3},
    }

    if args.dataset in dataset_T_map.keys():
        args.T = dataset_T_map[args.dataset]["T"]
        #args.moe_max_experts = dataset_T_map[args.dataset]["moe_max_experts"] #immer 5!
        print(f"Dataset {args.dataset} has T={args.T} and moe_max_experts={args.moe_max_experts}")

    return args




def calculate_similarity(feature1, feature2, metric='cosine'):
    """
    Calculates the similarity between two feature vectors.

    Args:
        feature1 (np.ndarray): First feature vector.
        feature2 (np.ndarray): Second feature vector.
        metric (str): The similarity metric to use ('cosine' or 'euclidean').

    Returns:
        float: The similarity score.
    """
    if metric == 'cosine':
        return cosine_similarity(feature1.reshape(1, -1), feature2.reshape(1, -1))[0][0]
    elif metric == 'euclidean':
        return -np.linalg.norm(feature1 - feature2) # Negative for consistency (higher value = more similar)
    else:
        raise ValueError(f"Unsupported similarity metric: {metric}")

def calculate_intra_class_similarity(features, labels, similarity_metric='cosine'):
    """
    Calculates the average intra-class similarity for a dataset.

    Args:
        features (np.ndarray): Array of feature vectors.
        labels (np.ndarray): Array of corresponding labels.
        similarity_metric (str): The similarity metric to use ('cosine' or 'euclidean').

    Returns:
        tuple: (dict, float) - A dictionary of per-class intra-class similarities
               and the overall average intra-class similarity.
    """
    intra_class_similarities = {}
    unique_labels = np.unique(labels)
    bar = tqdm(enumerate(unique_labels), desc="Calculating Intra-Class Similarity", total=len(unique_labels))
    for i, label in bar:
        class_features = features[labels == label]
        similarities = []
        for i in range(len(class_features)):
            for j in range(i + 1, len(class_features)):
                similarity = calculate_similarity(class_features[i], class_features[j], similarity_metric)
                similarities.append(similarity)
        if similarities:
            intra_class_similarities[label] = np.mean(similarities)
        else:
            intra_class_similarities[label] = 0.0

    overall_intra_class_similarity = np.mean(list(intra_class_similarities.values())) if intra_class_similarities else 0.0
    return intra_class_similarities, overall_intra_class_similarity

# new for intra... last chapter
def calculate_similarity(feature1: np.ndarray, feature2: np.ndarray, metric: str = 'cosine') -> float:
    """
    Calculates the similarity between two feature vectors.

    Args:
        feature1 (np.ndarray): First feature vector.
        feature2 (np.ndarray): Second feature vector.
        metric (str): The similarity metric to use ('cosine' or 'euclidean').

    Returns:
        float: The similarity score.
    """
    if metric == 'cosine':
        # Ensure inputs are 2D for scikit-learn's cosine_similarity
        return cosine_similarity(feature1.reshape(1, -1), feature2.reshape(1, -1))[0][0]
    elif metric == 'euclidean':
        # euclidean_distances returns distances, so we negate for similarity.
        # It's more efficient to calculate norm directly for two vectors.
        return -np.linalg.norm(feature1 - feature2)
    else:
        raise ValueError(f"Unsupported similarity metric: {metric}")

def calculate_intra_class_similarity_optimized(features: np.ndarray, labels: np.ndarray, similarity_metric: str = 'cosine') -> Tuple[dict, float]:
    """
    Calculates the average intra-class similarity for a dataset efficiently.

    Args:
        features (np.ndarray): Array of feature vectors (n_samples, n_features).
        labels (np.ndarray): Array of corresponding labels (n_samples,).
        similarity_metric (str): The similarity metric to use ('cosine' or 'euclidean').

    Returns:
        tuple: (dict, float) - A dictionary of per-class intra-class similarities
               and the overall average intra-class similarity.
    """
    intra_class_similarities = {}
    unique_labels = np.unique(labels)

    # Choose the appropriate pairwise function based on the metric
    if similarity_metric == 'cosine':
        pairwise_func = cosine_similarity
    elif similarity_metric == 'euclidean':
        # For Euclidean distance, we'll use euclidean_distances and then negate it
        pairwise_func = euclidean_distances
    else:
        raise ValueError(f"Unsupported similarity metric: {similarity_metric}")

    bar = tqdm(unique_labels, desc="Calculating Intra-Class Similarity", unit="class")
    for label in bar:
        class_features = features[labels == label]
        n_samples_in_class = class_features.shape[0]

        if n_samples_in_class < 2:
            # Cannot calculate pairwise similarity for 0 or 1 sample
            intra_class_similarities[label] = 0.0
            continue

        # Calculate all-to-all pairwise similarities within the class
        # This is where the main efficiency gain comes from
        pairwise_sims_or_dists = pairwise_func(class_features)

        # Extract only the upper triangle (excluding diagonal) to avoid duplicates and self-similarity
        # np.triu_indices_from gets indices for upper triangle
        upper_tri_indices = np.triu_indices_from(pairwise_sims_or_dists, k=1)
        
        similarities = pairwise_sims_or_dists[upper_tri_indices]

        # If Euclidean, negate the distances to represent similarity
        if similarity_metric == 'euclidean':
            similarities = -similarities
        
        if similarities.size > 0:
            intra_class_similarities[label] = np.mean(similarities)
        else:
            intra_class_similarities[label] = 0.0 # Should not happen if n_samples_in_class >= 2

    overall_intra_class_similarity = np.mean(list(intra_class_similarities.values())) if intra_class_similarities else 0.0
    
    return intra_class_similarities, overall_intra_class_similarity


def calculate_inter_class_similarity(features, labels, similarity_metric='cosine'):
    """
    Calculates the average inter-class similarity for a dataset.

    Args:
        features (np.ndarray): Array of feature vectors.
        labels (np.ndarray): Array of corresponding labels.
        similarity_metric (str): The similarity metric to use ('cosine' or 'euclidean').

    Returns:
        float: The overall average inter-class similarity.
    """
    inter_class_similarities = []
    unique_labels = np.unique(labels)
    bar = tqdm(enumerate(unique_labels), desc="Calculating Inter-Class Similarity", total=len(unique_labels))
    # Iterate through all pairs of classes
    for i, _ in bar:
        for j in range(i + 1, len(unique_labels)):
            label1 = unique_labels[i]
            label2 = unique_labels[j]
            features_class1 = features[labels == label1]
            features_class2 = features[labels == label2]
            for feat1 in features_class1:
                for feat2 in features_class2:
                    similarity = calculate_similarity(feat1, feat2, similarity_metric)
                    inter_class_similarities.append(similarity)

    overall_inter_class_similarity = np.mean(inter_class_similarities) if inter_class_similarities else 0.0
    return overall_inter_class_similarity

def calculate_inter_class_similarity_vectorized(features, labels, similarity_metric='cosine'):
    """
    Calculates the average inter-class similarity for a dataset using vectorization.
    """
    inter_class_similarities = []
    unique_labels = np.unique(labels)
    n_classes = len(unique_labels)
    bar = tqdm(range(n_classes), desc="Calculating Inter-Class Similarity", total=n_classes)

    for i in bar:
        for j in range(i + 1, n_classes):
            label1 = unique_labels[i]
            label2 = unique_labels[j]
            features_class1 = features[labels == label1]
            features_class2 = features[labels == label2]

            if similarity_metric == 'cosine':
                similarity_matrix = cosine_similarity(features_class1, features_class2)
                inter_class_similarities.extend(similarity_matrix.flatten())
            elif similarity_metric == 'euclidean':
                # Calculate pairwise Euclidean distances and negate for consistency
                distances = np.linalg.norm(features_class1[:, np.newaxis, :] - features_class2[np.newaxis, :, :], axis=2)
                inter_class_similarities.extend((-distances).flatten())
            else:
                raise ValueError(f"Unsupported similarity metric: {similarity_metric}")

    overall_inter_class_similarity = np.mean(inter_class_similarities) if inter_class_similarities else 0.0
    return overall_inter_class_similarity

# Expert learned a set of classes and we want to calculate the similarity to all other classes
def calculate_selective_inter_class_similarity(features, labels, target_classes, similarity_metric='cosine'):
    """
    Calculates the average inter-class similarity between a target list of classes
    and all other classes not in the target list, using vectorization.

    Args:
        features (np.ndarray): Array of feature vectors.
        labels (np.ndarray): Array of corresponding labels.
        target_classes (list): List of class labels for which to compute similarity
                               to all other classes (excluding those in this list).
        similarity_metric (str, optional): Metric to use for similarity calculation.
                                           'cosine' or 'euclidean' are supported.
                                           Defaults to 'cosine'.

    Returns:
        float: The average inter-class similarity between the target classes
               and the other classes. Returns 0.0 if no such pairs exist.
    """
    inter_class_similarities = []
    unique_labels = np.unique(labels)
    target_classes = set(target_classes)  # Convert to set for faster lookups
    other_classes = [label for label in unique_labels if label not in target_classes]

    bar = tqdm(target_classes, desc="Calculating Selective Inter-Class Similarity", total=len(target_classes))

    for label1 in bar:
        features_class1 = features[labels == label1]
        for label2 in other_classes:
            features_class2 = features[labels == label2]

            if similarity_metric == 'cosine':
                similarity_matrix = cosine_similarity(features_class1, features_class2)
                inter_class_similarities.extend(similarity_matrix.flatten())
            elif similarity_metric == 'euclidean':
                # Calculate pairwise Euclidean distances and negate for consistency
                distances = np.linalg.norm(features_class1[:, np.newaxis, :] - features_class2[np.newaxis, :, :], axis=2)
                inter_class_similarities.extend((-distances).flatten())
            else:
                raise ValueError(f"Unsupported similarity metric: {similarity_metric}")

    overall_inter_class_similarity = np.mean(inter_class_similarities) if inter_class_similarities else 0.0
    return overall_inter_class_similarity


def calculate_entropy(labels):
    """
    Calculates the entropy of a list or NumPy array of labels.

    Args:
        labels (list or np.ndarray): A list or array of labels.

    Returns:
        float: The entropy of the labels.
    """

    label_counts = Counter(labels)
    total_samples = len(labels)
    entropy = 0.0

    for count in label_counts.values():
        probability = count / total_samples
        entropy -= probability * math.log2(probability)

    return entropy

def calculate_feature_variance(features):
    """
    Calculates the variance of the features.

    Args:
        features (np.ndarray): Array of feature vectors.

    Returns:
        float: The variance of the features.
    """
    return np.var(features, axis=0).mean()  # Mean variance across all features

def visualize_csv_with_adjusted_size(csv_filepath, output_filepath="heatmap_adjusted.png"):
    try:
        df = pd.read_csv(csv_filepath, index_col=0)
    except FileNotFoundError:
        print(f"Fehler: Datei '{csv_filepath}' nicht gefunden.")
        return

    numeric_cols = df.select_dtypes(include=np.number).columns

    if not numeric_cols.empty:
        df_normalized = df[numeric_cols].apply(lambda x: (x - x.min()) / (x.max() - x.min()), axis=0)
        cmap = LinearSegmentedColormap.from_list("mycmap", ["white", "lightblue", "darkblue"])

        # Erhöhe die Figurengröße, um die Kästchen größer zu machen
        plt.figure(figsize=(len(numeric_cols) * 2, len(df) * 1))

        sns.heatmap(df_normalized, annot=False, cmap=cmap, cbar=True, yticklabels=True)
        plt.title("Farbliche Visualisierung der Datenspalten", fontsize=12) # Kleinere Schriftgröße für den Titel
        plt.xlabel("Numerische Spalten", fontsize=10) # Kleinere Schriftgröße für die X-Achse
        plt.ylabel("Datensätze", fontsize=10) # Kleinere Schriftgröße für die Y-Achse
        plt.xticks(rotation=45, ha="right", fontsize=8) # Kleinere Schriftgröße für die X-Achsenbeschriftungen
        plt.yticks(fontsize=8) # Kleinere Schriftgröße für die Y-Achsenbeschriftungen
        plt.tight_layout()
        plt.savefig(output_filepath)
        print(f"Heatmap mit angepasster Größe und Schrift gespeichert als '{output_filepath}'.")
        plt.close()
    else:
        print("Keine numerischen Spalten zum Visualisieren gefunden.")


def get_all_dataset_metrics(train_features, train_labels, dataset_name):
    values = [None] * 6
    values[0] = dataset_name
    
    label_entropy = calculate_entropy(train_labels)
    print(f"Label Entropy: {label_entropy:.4f}")
    values[1] = label_entropy
    feature_entropy = calculate_entropy(train_features.flatten())
    print(f"Feature Entropy: {feature_entropy:.4f}")
    values[2] = feature_entropy

    # 4. Calculate intra-class similarity on the training set
    intra_class_similarities, overall_intra_similarity = calculate_intra_class_similarity(train_features, train_labels) # Use train_labels here
    print(f"Intra-Class Similarities per class: {intra_class_similarities}")
    print(f"Overall Intra-Class Similarity: {overall_intra_similarity:.4f}")
    values[5] = overall_intra_similarity
    
    # 5. Calculate inter-class similarity on the training set
    overall_inter_similarity = calculate_inter_class_similarity_vectorized(train_features, train_labels) # Use train_labels here
    print(f"Overall Inter-Class Similarity: {overall_inter_similarity:.4f}")
    values[4] = overall_inter_similarity

    # 6. Calculate feature variance
    feature_variance = calculate_feature_variance(train_features)
    print(f"Feature Variance: {feature_variance:.4f}")
    values[3] = feature_variance

    return values


def main(args):
    # get dataset and augmentations
    train_transform = make_train_transform_from_args(args)
    test_transform = make_test_transform_from_args(args)
    train_base_dataset, test_base_dataset = get_dataset(
        args.dataset, path=args.data_root
    )
    update_transforms(test_base_dataset, transform=test_transform)


    # get datamanager based on ds
    data_manager = None
    if DILDataManager.is_dil(str(train_base_dataset)):
        print("DIL")
        data_manager = DILDataManager(
            train_base_dataset,
            test_base_dataset,
        )
    else:
        print("CIL")
        data_manager = CILDataManager(
            train_base_dataset,
            test_base_dataset,
            T=args.T,
            num_first_task=None if args.dataset != "cars" else 16,
            shuffle=True,
            seed=args.seed,
        )


    feature_extractor = timm.create_model(args.backbone, pretrained=True).to(args.device)
    feature_extractor.head = nn.Identity()
    feature_extractor.eval()
    train_features = []
    train_labels = []
    bar = tqdm(enumerate(data_manager), desc="Extracting Features", total=len(data_manager))
    for i, (train_dataset, _) in bar:
        train_dataset.transform = train_transform
        train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=False, num_workers=2, pin_memory=True)
        for images, labels in train_loader:  # Iterate through all batches in the loader
            images = images.to(args.device)
            features = feature_extractor(images)
            train_features.append(features.cpu().detach().numpy())
            train_labels.append(labels.cpu().detach().numpy())

    del feature_extractor

    train_features = np.concatenate(train_features, axis=0)
    train_labels = np.concatenate(train_labels, axis=0)
    print("Features shape:", train_features.shape)

    return train_features, train_labels   

    #values = get_all_dataset_metrics(train_features, train_labels, args.dataset)
    
"""
    # Saving values
    save_path = "local_data/dataset_metrics.csv"
    with open(save_path, 'a', newline='') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(values)
"""


def main2_classes_per_task_and_seed(args):
    # get dataset and augmentations
    train_transform = make_train_transform_from_args(args)
    test_transform = make_test_transform_from_args(args)
    train_base_dataset, test_base_dataset = get_dataset(
        args.dataset, path=args.data_root
    )
    update_transforms(test_base_dataset, transform=test_transform)


    # get datamanager based on ds
    data_manager = None
    if DILDataManager.is_dil(str(train_base_dataset)):
        print("DIL")
        data_manager = DILDataManager(
            train_base_dataset,
            test_base_dataset,
        )
    else:
        print("CIL")
        data_manager = CILDataManager(
            train_base_dataset,
            test_base_dataset,
            T=args.T,
            num_first_task=None if args.dataset != "cars" else 16,
            shuffle=True,
            seed=args.seed,
        )
        class_order = data_manager.class_order

    csv_rows = []
    bar = tqdm(enumerate(data_manager), desc="Accumulating classes per task", total=len(data_manager))
    for t, (train_dataset, _) in bar:
        task_labels_set = np.array([], dtype=np.int_)

        train_dataset.transform = train_transform
        train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=False, num_workers=2, pin_memory=True)
        for _, labels in train_loader:  # Iterate through all batches in the loader
            batch_labels = labels.cpu().detach().numpy()
            task_labels_set = np.concatenate((task_labels_set, batch_labels))
        # Get unique labels
        _, idx = np.unique(task_labels_set, return_index=True)
        task_labels_set = task_labels_set[np.sort(idx)]
        # label mapping
        for i, e in enumerate(task_labels_set):
            task_labels_set[i] = class_order[e]

        for i in task_labels_set:
            row = {
                "dataset": args.dataset,
                "task": t,
                "class": i,
                "seed": args.seed
            }
            csv_rows.append(row)




    # Save the results to a CSV file
    save_path = f"local_data/classes_per_task/CpT_{args.dataset}_{args.seed}.csv"
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    with open(save_path, 'w', newline='') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=["dataset", "task", "class", "seed"])
        writer.writeheader()
        writer.writerows(csv_rows)
    print(f"Classes per task saved in {save_path}")

In [9]:
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
args = parse_arguments()


seeds = [2000, 2001, 2002, 2003, 2004]
datasets = ["imagenetr", "cifar100", "cub", "imageneta", "cars", "omnibenchmark"]


for seed in seeds:
    for dataset in datasets:
        args.seed = seed
        args.dataset = dataset
        args.device = "cpu"
        args = update_args(args)

        set_seed(args.seed)

        main2_classes_per_task_and_seed(args)

CIL


Iterate over train dataset: 24000it [00:00, 663822.00it/s]
Iterate over test dataset: 6000it [00:00, 1166272.31it/s]
Accumulating classes per task: 100%|██████████| 10/10 [02:04<00:00, 12.46s/it]


Classes per task saved in local_data/classes_per_task/CpT_imagenetr_2000.csv
Files already downloaded and verified
Files already downloaded and verified
CIL


Iterate over train dataset: 50000it [00:02, 19682.41it/s]
Iterate over test dataset: 10000it [00:14, 691.54it/s]
Accumulating classes per task: 100%|██████████| 10/10 [01:27<00:00,  8.77s/it]


Classes per task saved in local_data/classes_per_task/CpT_cifar100_2000.csv
CIL


...

Classes per task saved in local_data/classes_per_task/CpT_imageneta_2004.csv
CIL


Iterate over train dataset: 8144it [00:00, 1320693.31it/s]
Iterate over test dataset: 8041it [00:00, 1280522.38it/s]
Accumulating classes per task: 100%|██████████| 10/10 [00:56<00:00,  5.65s/it]


Classes per task saved in local_data/classes_per_task/CpT_cars_2004.csv
CIL


Iterate over train dataset: 89697it [00:00, 943193.95it/s]
Iterate over test dataset: 5985it [00:00, 1110994.00it/s]
Accumulating classes per task: 100%|██████████| 10/10 [09:02<00:00, 54.28s/it]

Classes per task saved in local_data/classes_per_task/CpT_omnibenchmark_2004.csv


In [3]:
def parse_arguments():
    parser = argparse.ArgumentParser(description="Your script description here")

    # Define your arguments as before
    parser.add_argument('--batch_size', type=int, default=32, help='Batch size for training')
    parser.add_argument('--lr', type=float, default=0.001, help='Learning rate')
    parser.add_argument('--weight_decay', type=float, default=0.0, help='Weight decay')
    parser.add_argument('--early_stopping', type=int, default=10, help='Patience for early stopping')
    parser.add_argument('--dataset', type=str, default='cifar100',
                        choices=['cifar100', 'imagenetr', 'imageneta', 'vtab', 'cars', 'cub',
                                 'omnibenchmark', 'dil_imagenetr', 'cddb', 'limited_domainnet'],
                        help='Dataset to use')
    parser.add_argument('--T', type=int, default=10, help='Number of timesteps')
    parser.add_argument('--backbone', type=str, default='vit_base_patch16_224',
                        choices=['vit_base_patch16_224', 'vit_base_patch16_224_in21k'],
                        help='Backbone architecture')
    parser.add_argument('--finetune_method', type=str, default='none',
                        choices=['none', 'adapter', 'ssf', 'vpt'],
                        help='Finetuning method')
    parser.add_argument('--finetune_epochs', type=int, default=20, help='Number of finetuning epochs')
    parser.add_argument('--k', type=int, default=5, help='Number of nearest neighbors')
    parser.add_argument('--device', type=str, default='cuda' if 'cuda' in sys.modules else 'cpu',
                        help='Device to use (cuda or cpu)')
    parser.add_argument('--seed', type=int, default=2001, help='Random seed')
    parser.add_argument('--data_root', type=str, default='./local_data', help='Root directory for datasets')
    parser.add_argument('--moe_max_experts', type=int, default=4, help='Maximum number of experts for MoE')
    parser.add_argument('--reduce_dataset', type=float, default=1.0, help='Fraction of dataset to use')
    parser.add_argument('--gmms', type=int, default=8, help='Number of GMM components')
    parser.add_argument('--use_multivariate', action='store_true', help='Use multivariate Gaussian')
    parser.add_argument('--selection_method', type=str, default='random',
                        choices=['random', 'around', 'eucld_dist', 'inv_eucld_dist', 'kl_div',
                                 'inv_kl_div', 'ws_div', 'inv_ws_div'],
                        help='Selection method')
    parser.add_argument('--kd', action='store_true', help='Use knowledge distillation')
    parser.add_argument('--kd_alpha', type=float, default=0.5, help='Alpha for knowledge distillation loss')
    parser.add_argument('--log_gpustat', action='store_true', help='Log GPU statistics')
    parser.add_argument('--sweep_logging', action='store_true', help='Enable Weights & Biases sweep logging')
    parser.add_argument('--exit_after_T', action='store_true', help='Exit after T timesteps')
    parser.add_argument('--selection_criterion', type=int, default=0, choices=[0, 1, 2],
                        help='Selection criterion')
    parser.add_argument('--tau', type=float, default=0.1, help='Temperature parameter')
    parser.add_argument('--exit_after_acc', type=float, default=0.0, help='Exit after reaching this accuracy')
    parser.add_argument('--trash_var', type=float, default=0.0, help='Trash variable (for testing)')
    parser.add_argument('--use_adamw', action='store_true', help='Use AdamW optimizer')
    parser.add_argument('--use_cosine_annealing', action='store_true', help='Use cosine annealing scheduler')
    parser.add_argument('--aug_resize_crop_min', type=float, default=0.8, help='Min scale for random resize crop')
    parser.add_argument('--aug_resize_crop_max', type=float, default=1.0, help='Max scale for random resize crop')
    parser.add_argument('--aug_random_rotation_degree', type=float, default=0.0, help='Degree for random rotation')
    parser.add_argument('--aug_brightness_jitter', type=float, default=0.0, help='Brightness jitter')
    parser.add_argument('--aug_contrast_jitter', type=float, default=0.0, help='Contrast jitter')
    parser.add_argument('--aug_saturation_jitter', type=float, default=0.0, help='Saturation jitter')
    parser.add_argument('--aug_hue_jitter', type=float, default=0.0, help='Hue jitter')
    parser.add_argument('--aug_normalize', action='store_true', help='Normalize input images')
    parser.add_argument('--wandb_project', type=str, default='your_project_name', help='WandB project name')
    parser.add_argument('--wandb_entity', type=str, default='your_entity_name', help='WandB entity name')

    if '__file__' in globals():  # Check if running as a script
        args = parser.parse_args()
    else:  # Running in a Jupyter Notebook
        args = parser.parse_args(args=[]) # Pass an empty list to avoid errors
        # You can set default values here or handle arguments differently in the notebook

    return args

In [8]:
os.environ['CUDA_VISIBLE_DEVICES'] = CUDA_VISIBLE_DEVICES
args = parse_arguments()
args.dataset = DATASET
args = update_args(args)
set_seed(args.seed)


features, labels = main(args)

CIL


Iterate over train dataset: 1796it [00:00, 1226069.33it/s]
Iterate over test dataset: 8619it [00:00, 1487438.54it/s]
Extracting Features: 100%|██████████| 5/5 [30:13<00:00, 362.69s/it]

Features shape: (1796, 768)


## Saving all features into file 

In [ ]:
dataset_name = DATASET
labels_list = labels
features_list = features

# Stelle sicher, dass features_list eine Python-Liste ist
if not isinstance(features_list, list):
    features_list = features_list.tolist()  # Konvertiere zu Liste, falls es ein NumPy Array ist

# Annahme: Jede innere Liste hat die gleiche Länge und entspricht einer Zeile
# und die Elemente der inneren Liste sollen separate Spalten werden.

# Erstelle Spaltennamen für die Features
num_features = len(features_list[0]) if features_list else 0
feature_columns = [f'feature_{i}' for i in range(num_features)]

# Erstelle ein Dictionary für den DataFrame
data = {'dataset': [dataset_name] * len(labels_list),
        'label': [item[0] if isinstance(item, list) else item for item in labels_list]} # Annahme: Label ist das erste Element der inneren Liste
for i, col in enumerate(feature_columns):
    data[col] = [item[i] if isinstance(item, list) and len(item) > i else None for item in features_list]

df = pd.DataFrame(data)

# Speichern als CSV-Datei
csv_filename = f"./local_data/{dataset_name}_class_features.csv"
df.to_csv(csv_filename, index=False)
print(f"Daten erfolgreich als '{csv_filename}' gespeichert.")


Daten erfolgreich als './local_data/vtab_class_features.csv' gespeichert (korrigiert).


## Rename class labels
- Die Labels werden ranomised nach seed, werden im Modell aber immer normal durchnummeriert. Das führt aber zu einem missmatch mit den erstellten Features, weil die eine ganz bestimmte random Nummerierung haben, aber 0-99 gelabelt werden.  
- Die Labelreihenfolge wurde bei DIL nicht gelogged. Ich beschränke mich auf CIL!
- Features werden neu erstellt. Mit shuffle=False. Mit Logs (class order =>) überprüfen

In [ ]:
def get_class_order_from_logs(run_id):
    """Lädt die Log-Datei eines WandB-Runs herunter und gibt die Zeile mit "class order => ..." zurück.

    Args:
        run_id (str): Die ID des WandB-Runs.

    Returns:
        str: Die Zeile, die mit "class order => ..." beginnt, oder None, falls nicht gefunden.
    """
    api = wandb.Api()
    try:
        run = api.run(run_id)  # Ersetze mit deiner Entity und deinem Projekt
        if run.files:
            for f in run.files():
                if f.name == 'output.log':  # Der typische Name für die stdout/stderr Logs
                    log_content = f.download(root='.', replace=True, exist_ok=True)
                    if log_content:
                        with open(log_content.name, 'r') as log_file:
                            for line in log_file:
                                if line.startswith("class_order =>"):
                                    print(f"Gefundene Zeile: {line.strip()}")
                                    os.remove(log_content.name) # Optional: Lösche die temporäre Log-Datei
                                    return line.strip()                                
                        os.remove(log_content.name) # Optional: Lösche die temporäre Log-Datei
                        print(f"Keine Zeile mit 'class order => ...' in der Log-Datei für Run {run_id} gefunden.")
                        return None
        print(f"Keine 'output.log'-Datei für Run {run_id} gefunden.")
        return None
    except wandb.CommError as e:
        print(f"Fehler beim Abrufen des Runs {run_id}: {e}")
        return None

# Beispielhafte Verwendung
run_path = "belaschindler-university-hamburg/0schindl-LayUp_sweeps_question1/runs/crnqagtv"  # Ersetze mit der tatsächlichen Run-ID
class_order_line = get_class_order_from_logs(run_path)

if class_order_line:
    print(f"Die Zeile mit 'class order => ...' ist: {class_order_line}")
else:
    print("Die Zeile mit 'class order => ...' wurde nicht gefunden.")

class_order_line = np.array(class_order_line.lstrip("class_order =>"))


Gefundene Zeile: class_order => [164, 106, 3, 122, 90, 148, 76, 193, 179, 177, 142, 192, 6, 43, 131, 30, 104, 20, 160, 115, 80, 69, 119, 55, 99, 78, 172, 185, 145, 8, 85, 47, 154, 13, 88, 195, 19, 178, 132, 51, 81, 70, 170, 87, 184, 113, 14, 7, 125, 117, 139, 79, 50, 107, 114, 97, 92, 174, 77, 41, 54, 150, 108, 35, 156, 149, 56, 130, 110, 40, 135, 196, 17, 91, 10, 173, 37, 22, 32, 100, 189, 12, 116, 66, 199, 71, 23, 161, 136, 94, 36, 158, 52, 67, 27, 187, 153, 63, 98, 33, 45, 74, 18, 93, 140, 59, 190, 165, 83, 15, 191, 5, 157, 123, 57, 138, 144, 53, 95, 166, 65, 49, 38, 25, 58, 11, 163, 75, 152, 89, 96, 159, 64, 34, 167, 168, 2, 176, 103, 102, 124, 29, 28, 73, 24, 180, 162, 84, 31, 197, 137, 126, 182, 133, 111, 61, 46, 155, 181, 188, 127, 21, 82, 169, 9, 143, 48, 109, 120, 183, 194, 146, 62, 112, 118, 129, 151, 175, 141, 86, 105, 72, 147, 39, 198, 121, 26, 44, 60, 171, 68, 42, 128, 1, 101, 186, 16, 134, 4, 0]
Die Zeile mit 'class order => ...' ist: class_order => [164, 106, 3, 122, 90,

## Load data from file

In [6]:
csv_filename = f"./local_data/class_features/class_features_{DATASET}.csv"

# Lese die CSV-Datei ein
df_loaded = pd.read_csv(csv_filename)

# Extrahiere die Labels und Features
loaded_labels = df_loaded['label'].tolist()

# Wenn die Features als separate Spalten gespeichert wurden (Szenario 1):
if 'feature_0' in df_loaded.columns:
    loaded_features = []
    i = 0
    while f'feature_{i}' in df_loaded.columns:
        loaded_features.append(df_loaded[f'feature_{i}'].tolist())
        i += 1
    # Transponiere die Liste von Listen, um die ursprüngliche Struktur wiederherzustellen
    loaded_features = list(zip(*loaded_features))
    loaded_features = [list(item) for item in loaded_features]

# Wenn die Features als Listen in einer Zelle gespeichert wurden (Szenario 2):
elif 'feature' in df_loaded.columns:
    loaded_features = [ast.literal_eval(item) for item in df_loaded['feature'].tolist()]

print("Daten erfolgreich aus CSV eingelesen:")
print("Geladene Labels (erste 5):", loaded_labels[:5])
print("Geladene Features (erste 1):", loaded_features[:1])


Daten erfolgreich aus CSV eingelesen:
Geladene Labels (erste 5): [0, 1, 8, 8, 9]
Geladene Features (erste 1): [[1.1573697328567505, 0.3032321631908417, 0.4962291121482849, 0.1067860126495361, -1.625678539276123, 0.4844562709331512, 0.6799285411834717, 0.4303563237190246, -1.5096973180770874, -1.3390100002288818, -0.6796276569366455, 0.913780689239502, 0.3822452127933502, 1.5332729816436768, 0.0255593322217464, -3.4735898971557617, 0.6965407133102417, 0.8458954691886902, 0.5373067855834961, -0.2713418304920196, 1.312721848487854, 4.076414585113525, -0.0822739154100418, 0.0968436002731323, -1.4931050539016724, -2.24968695640564, -0.820317804813385, -0.6016552448272705, 4.658712387084961, 0.5360066294670105, 0.9967613816261292, -0.3212272822856903, 1.1852117776870728, -1.3273863792419434, -2.471297264099121, -1.567915439605713, -0.9555495381355286, -1.0097880363464355, -0.481315404176712, 0.1386799365282058, 1.704442024230957, -1.3899831771850586, -1.121779203414917, 0.5226563811302185, -

## What are the classes per task?
Ich brauche eine übersetzung.  
Zusätzlich: Was ist mit den Seeds? Classenreihenfolge ist seed abhängig! Seed der runs berücksichtigen.

In [7]:
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
args = parse_arguments()


seeds = [2000, 2001, 2002, 2003, 2004]
datasets = ["imagenetr", "cifar100", "cub", "dil_imagenetr", "imageneta", "cars", "omnibenchmark", "limited_domainnet"]


for seed in seeds:
    for dataset in datasets:
        args.seed = seed
        args.dataset = dataset
        args.device = "cpu"
        args = update_args(args)

        set_seed(args.seed)

        main2_classes_per_task_and_seed(args)
        break
    break

CIL


Iterate over train dataset: 24000it [00:00, 977436.92it/s]
Iterate over test dataset: 6000it [00:00, 1569920.40it/s]
Accumulating classes per task: 100%|██████████| 10/10 [02:14<00:00, 13.44s/it]

Classes per task saved in local_data/classes_per_task/CpT_imagenetr_2000.csv


## Which expert learned which class

In [84]:
wandb.login(key="8a88a8c49d1c2d31b8677fe0b8eb7d3e3a031f83")
api = wandb.Api()


def get_expert_distribution(run):
    if run.state != "finished":
        return None

    history = run.history()
    
    expert_distributions = dict()
    ft_tasks = [None] * 1000
    ft_buffer = run.config.get("moe_max_experts")
    for line in run.history().columns:
        if line.startswith("Expert") and line.endswith("learned task"):
            
            line_splited = line.split(" ")
            expert = int(line_splited[1])
            tasks = history[line].dropna().tolist()
            tasks = [int(task) for task in tasks]
            
            if expert not in expert_distributions:
                expert_distributions[expert] = list()
            expert_distributions[expert].extend(tasks) 

            
            for i in tasks:
                if i >= ft_buffer:
                    ft_tasks[i - ft_buffer] = expert    

    # cleaning ft_tasks
    ft_tasks = [i for i in ft_tasks if i is not None]

    return ft_tasks, expert_distributions

def get_sweep_data(runs, attributes_config=[], attributes_summary=[], include_run_id=True, class_similarity=False):
    sweep_data = []
    for run in runs:
        config = run.config
        summary = run.summary

        if summary.get("task_mean/acc") is not None and run.state == "finished":
            run_data = dict()
            if include_run_id:
                run_data["run_id"] = run.id
            # Add the config attributes to the run_data dictionary
            for attr in attributes_config:
                run_data[attr] = config.get(attr)

            # Add the summary attributes to the run_data dictionary
            for attr in attributes_summary:
                run_data[attr] = summary.get(attr)
            
            # average class similarity of learned classes per expert
            if class_similarity and run_data["dataset"] == DATASET:
                _, task_distribution = get_expert_distribution(run)
                print(f"Run {run.id} - Task Distribution: {task_distribution}")

                # Map classes per task
                classes_per_task = get_classes_per_task(run_data["dataset"], run_data["seed"])
                for expert_id, learned_tasks in task_distribution.items():
                    classes= []
                    for task in learned_tasks:
                        classes.extend(classes_per_task[task])
                    task_distribution[expert_id] = classes
            

                # Calculate inter-class similarity for each expert
                expert_similaritys = list()
                for expert_id, learned_classes in task_distribution.items():
                    if len(learned_classes) > 1:
                        label_mask = np.isin(loaded_labels, learned_classes)

                        # Verwende die Maske, um die entsprechenden Features und Labels auszuwählen
                        expert_features = loaded_features[label_mask]
                        expert_labels = loaded_labels[label_mask]
                        print(expert_labels)
                        print(f"Expert {expert_id} learned classes: {learned_classes}")
                        print(f"Shape: {expert_features.shape}")
                        similarity = calculate_intra_class_similarity_optimized(expert_features, expert_labels, similarity_metric='cosine')[1]
                        print(f"Expert {expert_id} - Intra-Class Similarity: {similarity:.4f}")
                        expert_similaritys.append(similarity)

                run_data["expert_similaritys"] = expert_similaritys
                average_similarity = np.mean(expert_similaritys) if expert_similaritys else 0.0       
                run_data["average_expert_similarity"] = average_similarity
                print(f"Average Expert Similarity: {average_similarity:.4f}")




            sweep_data.append(run_data)

    return sweep_data


def get_classes_per_task(dataset, seed):
    """
    Get the classes per task for a given dataset and seed.
    """
    csv_path = f"local_data/classes_per_task/CpT_{dataset}_{seed}.csv"
    if not os.path.exists(csv_path):
        print(f"CSV file {csv_path} does not exist.")
        return None

    df = pd.read_csv(csv_path)
    classes_per_task = {}
    for _, row in df.iterrows():
        task = row['task']
        class_label = row['class']
        if task not in classes_per_task:
            classes_per_task[task] = []
        classes_per_task[task].append(class_label)

    return classes_per_task

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /export/home/0schindl/.netrc


## Average similarity per expert

In [105]:
DATASET = "omnibenchmark"

In [106]:
## Load data from file
csv_filename = f"./local_data/class_features/class_features_{DATASET}.csv"

# Lese die CSV-Datei ein
df_loaded = pd.read_csv(csv_filename)

# Extrahiere die Labels und Features
loaded_labels = df_loaded['label'].tolist()

# Wenn die Features als separate Spalten gespeichert wurden (Szenario 1):
if 'feature_0' in df_loaded.columns:
    loaded_features = []
    i = 0
    while f'feature_{i}' in df_loaded.columns:
        loaded_features.append(df_loaded[f'feature_{i}'].tolist())
        i += 1
    # Transponiere die Liste von Listen, um die ursprüngliche Struktur wiederherzustellen
    loaded_features = list(zip(*loaded_features))
    loaded_features = [list(item) for item in loaded_features]

# Wenn die Features als Listen in einer Zelle gespeichert wurden (Szenario 2):
elif 'feature' in df_loaded.columns:
    loaded_features = [ast.literal_eval(item) for item in df_loaded['feature'].tolist()]

print("Daten erfolgreich aus CSV eingelesen:")
print("Geladene Labels (erste 5):", loaded_labels[:5])
print("Geladene Features (erste 1):", loaded_features[:1])


Daten erfolgreich aus CSV eingelesen:
Geladene Labels (erste 5): [0, 0, 0, 0, 0]
Geladene Features (erste 1): [[1.9689552783966064, 0.0870765522122383, -0.0113297663629055, 0.6839339137077332, 0.1754369139671325, 0.9810996055603028, -0.4749040901660919, 1.8897793292999268, -0.4923095107078552, -0.4473477303981781, -1.486798882484436, 0.8747390508651733, -0.0753733217716217, -0.6563200950622559, 0.718726634979248, -0.4465139806270599, 1.195293664932251, 0.6493381857872009, 0.6927939653396606, -0.5141491293907166, 0.8577017188072205, -0.3272809982299804, -1.0333484411239624, 0.4337775409221649, -1.115162372589111, -0.0877170041203498, 1.3080565929412842, 0.7482463121414185, -0.8268628716468811, -0.1584079265594482, -0.5005767941474915, -1.4997069835662842, 2.248469591140747, -1.0296906232833862, -0.1423937827348709, -0.4433310627937317, -0.1288578808307647, 0.3147761523723602, -0.3346352875232696, -0.6630178689956665, -0.0858169347047805, -0.3058180212974548, -0.4738557934761047, -0.5707

In [107]:

_42_adapter_performance = [
    "belaschindler-university-hamburg/0schindl-LayUp_sweeps_question1_results/6kim8tiu", # DIL
    "belaschindler-university-hamburg/0schindl-LayUp_sweeps_question1_results/jdpa9z1x", # Cars
    "belaschindler-university-hamburg/0schindl-LayUp_sweeps_question1_results/cjddpel4", # Imagenet-a
    "belaschindler-university-hamburg/0schindl-LayUp_sweeps_question1_results/hxigp6ck", # Imagenet-r
    "belaschindler-university-hamburg/0schindl-LayUp_sweeps_question1_results/p7zmthx9", # CIL
    ]
datsets_in_CIL = ["cifar100", "cub", "vtab", "omnibenchmark"]

loaded_labels = np.array(loaded_labels)
loaded_features = np.array(loaded_features)

table_421 = []
for i, s in enumerate(_42_adapter_performance):
    sweep = api.sweep(s)
    runs = sweep.runs

    attributes_config = ["dataset", "selection_method", "seed"]
    attributes_summary = ["task_mean/acc"]

    data = get_sweep_data(runs, attributes_config, attributes_summary, class_similarity=True)
    for e in data:
        if e["dataset"] == DATASET:
            if i == 4:
                # CIL
                if e["dataset"] in datsets_in_CIL:
                    table_421.append(e)
            else:
                table_421.append(e)
    


df_sweep = pd.DataFrame(table_421)

Run 6dwyk5lm - Task Distribution: {4: [4, 5, 7], 0: [0, 6], 2: [2, 8, 9], 1: [1], 3: [3]}
[  2   2   2 ... 299 299 299]
Expert 4 learned classes: [2, 13, 49, 53, 61, 65, 66, 73, 75, 77, 81, 84, 94, 100, 102, 113, 128, 145, 154, 171, 199, 216, 224, 229, 243, 258, 277, 285, 288, 290, 6, 26, 29, 30, 39, 51, 103, 105, 110, 116, 117, 121, 142, 152, 172, 181, 182, 190, 198, 211, 215, 218, 221, 223, 235, 250, 257, 265, 279, 294, 10, 12, 22, 23, 55, 69, 76, 83, 93, 101, 140, 160, 161, 167, 168, 170, 176, 209, 233, 236, 244, 263, 269, 271, 280, 286, 289, 292, 293, 299]
Shape: (26911, 768)


Calculating Intra-Class Similarity: 100%|██████████| 90/90 [00:00<00:00, 260.36class/s]


Expert 4 - Intra-Class Similarity: 0.5415
[  5   5   5 ... 297 297 297]
Expert 0 learned classes: [5, 17, 38, 50, 58, 67, 70, 74, 89, 99, 106, 114, 118, 119, 122, 130, 136, 137, 151, 166, 177, 197, 205, 210, 219, 220, 248, 252, 270, 297, 8, 16, 31, 34, 36, 42, 48, 60, 82, 86, 92, 104, 112, 120, 126, 138, 139, 143, 148, 158, 165, 185, 200, 201, 202, 228, 241, 275, 287, 296]
Shape: (18264, 768)


Calculating Intra-Class Similarity: 100%|██████████| 60/60 [00:00<00:00, 243.82class/s]


Expert 0 - Intra-Class Similarity: 0.5497
[  1   1   1 ... 298 298 298]
Expert 2 learned classes: [32, 35, 57, 64, 68, 80, 95, 111, 115, 125, 131, 132, 146, 150, 155, 179, 187, 188, 193, 196, 212, 231, 232, 238, 240, 253, 262, 266, 267, 273, 7, 9, 21, 24, 37, 40, 44, 62, 78, 79, 85, 96, 97, 107, 134, 135, 141, 159, 174, 184, 208, 226, 242, 246, 256, 259, 260, 276, 278, 291, 1, 4, 14, 15, 28, 41, 43, 52, 54, 63, 71, 91, 109, 123, 124, 164, 178, 183, 192, 194, 206, 207, 214, 217, 247, 264, 268, 272, 284, 298]
Shape: (26990, 768)


...

Expert 1 - Intra-Class Similarity: 0.5414
[  1   1   1 ... 296 296 296]
Expert 2 learned classes: [1, 18, 21, 36, 38, 55, 70, 77, 78, 81, 85, 88, 91, 92, 112, 113, 145, 158, 167, 182, 225, 227, 234, 243, 248, 258, 259, 261, 264, 294, 2, 25, 34, 64, 65, 75, 89, 95, 96, 109, 139, 152, 166, 176, 181, 185, 204, 212, 221, 231, 242, 254, 266, 272, 273, 284, 287, 290, 293, 296, 24, 28, 29, 31, 72, 73, 82, 84, 102, 103, 111, 124, 126, 127, 133, 134, 137, 143, 155, 174, 206, 211, 230, 235, 240, 249, 265, 283, 285, 286, 4, 16, 26, 39, 42, 48, 60, 62, 86, 101, 105, 120, 121, 128, 129, 141, 146, 147, 175, 177, 186, 197, 208, 224, 250, 253, 256, 257, 260, 295]
Shape: (35984, 768)


Calculating Intra-Class Similarity: 100%|██████████| 120/120 [00:00<00:00, 256.68class/s]


Expert 2 - Intra-Class Similarity: 0.5348
[  8   8   8 ... 255 255 255]
Expert 0 learned classes: [8, 9, 30, 32, 33, 41, 46, 49, 56, 66, 97, 98, 125, 150, 156, 157, 159, 165, 168, 178, 187, 192, 193, 194, 200, 209, 215, 244, 252, 255]
Shape: (8635, 768)


Calculating Intra-Class Similarity: 100%|██████████| 30/30 [00:00<00:00, 270.10class/s]

Expert 0 - Intra-Class Similarity: 0.5143
Average Expert Similarity: 0.5353


In [108]:
data = df_sweep
print(data)

     run_id        dataset selection_method  seed  task_mean/acc  \
0  6dwyk5lm  omnibenchmark       inv_ws_div  2000       0.573589   
1  7z247qq2  omnibenchmark       inv_ws_div  2004       0.581586   
2  5b5h2uzl  omnibenchmark       inv_ws_div  2003       0.597296   
3  fs3coo40  omnibenchmark       inv_ws_div  2002       0.585258   
4  26v6mgz7  omnibenchmark       inv_ws_div  2001       0.591332   

                                  expert_similaritys  \
0  [0.5415225760440951, 0.5496745796154197, 0.537...   
1  [0.4892236990547385, 0.5505489179248585, 0.536...   
2  [0.4966600350786431, 0.5529010815099288, 0.557...   
3  [0.5428289046490888, 0.5117119500755918, 0.587...   
4  [0.5483583799959261, 0.5374967049866456, 0.541...   

   average_expert_similarity  
0                   0.531002  
1                   0.533287  
2                   0.533252  
3                   0.542283  
4                   0.535253  


### CARS

   average_expert_similarity  
0                   0.673711  
1                   0.666535  
2                   0.668896  
3                   0.672675  
4                   0.666224  

### CIFAR
0                   0.477616  
1                   0.471247  
2                   0.473173  
3                   0.477304  
4                   0.474774  


### CUB
0                   0.629865  
1                   0.636929  
2                   0.630805  
3                   0.623717  
4                   0.638992  

### IN-A
0                   0.179068  
1                   0.181273  
2                   0.184007  
3                   0.185891  
4                   0.184179  

### IN-R
0                   0.268898  
1                   0.274224  
2                   0.276426  
3                   0.280183  
4                   0.278103  

## OB
0                   0.531002  
1                   0.533287  
2                   0.533252  
3                   0.542283  
4                   0.535253  



In [ ]:

                # Calculate inter-class similarity for each expert
                expert_similaritys = list()
                for expert_id, learned_classes in task_distribution.items():
                    if len(learned_classes) > 1:
                        label_mask = np.isin(loaded_labels, learned_classes)

                        # Verwende die Maske, um die entsprechenden Features und Labels auszuwählen
                        expert_features = loaded_features[label_mask]
                        expert_labels = loaded_labels[label_mask]
                        print(f"Expert {expert_id} learned classes: {learned_classes}")
                        print(f"Shape: {expert_features.shape}")
                        similarity = calculate_inter_class_similarity_vectorized(expert_features, expert_labels)
                        print(f"Expert {expert_id} - Inter-Class Similarity: {similarity:.4f}")
                        expert_similaritys.append(similarity)

                run_data["expert_similaritys"] = expert_similaritys
                average_similarity = np.mean(expert_similaritys) if expert_similaritys else 0.0       
                run_data["average_expert_similarity"] = average_similarity
                print(f"Average Expert Similarity: {average_similarity:.4f}")
                print(task_distribution)


                # inter-class similarity to all classes except the one where their expert only learned this one class
                lonly_learned_classes = []
                for expert_id, learned_classes in task_distribution.items():
                    if len(learned_classes) == 1:
                        lonly_learned_classes.append(learned_classes[0])
                print(f"Classes where expert only learned this class: {lonly_learned_classes}")

                # Calculate inter-class similarity to all other classes
                other_classes = [label for label in loaded_labels if label != lonly_learned_classes]
                label_mask = np.isin(loaded_labels, other_classes)

                expert_features = loaded_features[label_mask]
                expert_labels = loaded_labels[label_mask]
                similarity = calculate_inter_class_similarity_vectorized(expert_features, expert_labels)
                run_data["filtered_dataset_similarity"] = similarity
                print(f"Dataset similarity: {similarity:.4f}")


